In [2]:
import numpy as np
import csv
import cv2
import os
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D, Lambda
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import  np_utils
from keras.preprocessing.image import  ImageDataGenerator
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, auc

In [3]:
num_classes = 5
epochs = 50
BASE_DIR = '../'
batch_size = 32

In [4]:
def get_data(folder):
    X = []
    y = []
    
    for cyte in os.listdir(folder):
        if not cyte.startswith('.'):
            for image_filename in os.listdir(folder + cyte):
                img_file = cv2.imread(folder + cyte + '/' + image_filename)
                if img_file is not None:
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(cyte)
    
    X = np.asarray(X)
    y = np.asarray(y)
    return X, y

In [ ]:
X_train, y_train = get_data(BASE_DIR + "/content/drive/MyDrive/WBC_classification/Data/TRAIN/")